In [11]:
#多路召回，对于多种方法的结果进行加权比较
import json
bge = json.load(open("submit_bge_t10.json",encoding='utf-8'))
bm25 = json.load(open("submit_bm25_t10.json",encoding='utf-8'))

fusion_result = []
k=60
#进行排序的方式打分
for q1,q2 in zip(bge,bm25):
    fusion_score = {}
    for idx,q in enumerate(q1['reference']):
        if q not in fusion_score:
            fusion_score[q] = 1/ (idx + k)
        else:
            fusion_score[q] += 1 / (idx + k)

    for idx,q in enumerate(q2['reference']):
        if q not in fusion_score:
            fusion_score[q] = 1/ (idx + k)
        else:
            fusion_score[q] += 1 / (idx + k)

    sorted_dict = sorted(fusion_score.items(), key=lambda item:item[1], reverse=True)
    q1['reference'] = sorted_dict[0][0]
    fusion_result.append(q1)

with open('submit_fusion_ge+bm25.json','w',encoding='utf8') as up:
    json.dump(fusion_result, up, ensure_ascii=False, indent=4)

In [ ]:
import jieba, json, pdfplumber
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from rank_bm25 import BM25Okapi

questions = json.load(open("questions.json",encoding='utf-8'))

pdf = pdfplumber.open("初赛训练数据集.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })

# 加载重排序模型
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')
rerank_model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')
rerank_model.cuda()

pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

for query_idx in range(len(questions)):
		# 首先进行BM25检索
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idxs = doc_scores.argsort()[-3:]
		
		# top3进行重排序
    pairs = []
    for idx in max_score_page_idxs:
        pairs.append([questions[query_idx]["question"], pdf_content[idx]['content']])
     #统一编码
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        inputs = {key: inputs[key].cuda() for key in inputs.keys()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
        #打分
    max_score_page_idx = max_score_page_idxs[scores.cpu().numpy().argmax()]
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)

with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf-mirror.com/repos/fb/da/fbda43f02a4febe7d5ab4fa7e76c982ad061eefdb0d1a3eb08f1d684a1992954/ced967c45fd1902eb92716c9ceeca7c95a936770ea9db611f5a841b926e33fbd?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1720255691&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMDI1NTY5MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mYi9kYS9mYmRhNDNmMDJhNGZlYmU3ZDVhYjRmYTdlNzZjOTgyYWQwNjFlZWZkYjBkMWEzZWIwOGYxZDY4NGExOTkyOTU0L2NlZDk2N2M0NWZkMTkwMmViOTI3MTZjOWNlZWNhN2M5NWE5MzY3NzBlYTlkYjYxMWY1YTg0MWI5MjZlMzNmYmQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=lIoY5Kg6Y3ufOXoXae9E7uveuHbuuT1jTI%7EuiFS9PNn2EH4n6Ft2cpKIALkxEB9rwDpdsln9Gx5ysU03-FT8xuiVEUDbu9p2-VjAQGXhJJI-KW1n31wRQxGk-oWt7-CkFDvSqItrZcIz1jRiYju3-X7rd1uXbb1H74SMvdbQcrM8gBqTRyWlCLQvSDUEx-QZ%7EPHOgmP5U%7Erx8fZeA-ZsIN1MnCOqGaPK855zwduz-wWWLf2lZ

model.safetensors:   8%|7         | 83.9M/1.11G [00:00<?, ?B/s]